# Factory Extraction
> Functions to extract BaseFactory instances from modules:

In [ ]:
#| default_exp cli.factory_extraction

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass
from cjm_fasthtml_tailwind.core.base import BaseFactory
from typing import Dict, List, Tuple, Any, Optional

from cjm_fasthtml_tailwind.cli.utils import iterate_all_modules_with_items, discover_utility_modules

In [ ]:
#| export
@dataclass
class FactoryInfo:
    """Information about a discovered factory instance."""
    name: str  # Factory variable name (e.g., 'p', 'w', 'flex')
    factory: BaseFactory  # The actual factory instance
    doc: str  # Documentation from the factory
    module_name: str  # Module where it was found

In [ ]:
#| export
def extract_factories_from_module(
    module: Any,  # The module to extract factories from
    module_name: str  # The name of the module
) -> List[FactoryInfo]:  # List of FactoryInfo objects
    """Extract all BaseFactory instances from a module."""
    factories = []
    
    # Get all attributes of the module
    for name in dir(module):
        # Skip private attributes
        if name.startswith('_'):
            continue
            
        try:
            attr = getattr(module, name)
            
            # Check if it's an instance of BaseFactory (or its subclasses)
            if isinstance(attr, BaseFactory):
                # Get documentation
                doc = attr.describe() if hasattr(attr, 'describe') else str(attr.__doc__ or "")
                
                factories.append(FactoryInfo(
                    name=name,
                    factory=attr,
                    doc=doc,
                    module_name=module_name
                ))
        except AttributeError:
            pass  # Skip attributes that can't be accessed
    
    return sorted(factories, key=lambda x: x.name)  # Sort by factory name

In [ ]:
#| export
def list_all_factories(
) -> Dict[str, List[FactoryInfo]]:  # Dictionary mapping module names to their factories
    """List all factory instances across all utility modules."""
    return iterate_all_modules_with_items(extract_factories_from_module)

In [ ]:
#| export
def list_module_factories(
    module_name: str  # Name of the module to inspect (e.g., 'spacing', 'sizing')
) -> List[FactoryInfo]:  # List of FactoryInfo objects for the module
    """List all factory instances in a specific utility module."""
    # Find the module
    for name, module in discover_utility_modules():
        if name == module_name:
            return extract_factories_from_module(module, module_name)
    
    return []  # Module not found

In [ ]:
#| export
def get_factory_by_name(
    module_name: str,  # Name of the module
    factory_name: str  # Name of the factory (e.g., 'p', 'w', 'flex')
) -> Optional[FactoryInfo]:  # FactoryInfo object or None if not found
    """Get a specific factory by module name and factory name."""
    factories = list_module_factories(module_name)
    
    for factory_info in factories:
        if factory_info.name == factory_name:
            return factory_info
    
    return None

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()